# Light-Weight CNN LB 0.74
https://www.kaggle.com/alphasis/light-weight-cnn-lb-0-74

## 들어가며
이 노트북은 경량 CNN을 구축하는 것을 목표로 합니다.

리샘플링된 wav 파일의 (rate 8000) 스펙그램을 입력으로 사용합니다.

kaggle 클라우드 하드웨어 제한으로 인해 이 스크립트는 원래 스크립트의 '파손'된 버전입니다.

LB 0.74를 가져오려면 에폭을 5로 설정하고 chop_audio(num=1000)을 설정하고 모든 Conv 레이어 파라미터를 두 배로 설정해야 합니다.

이 스크립트는 Alex Ozerin의 기준보다 약간 개선되었지만, 원본 wav파일(16000 sample rate)을 사용하면 더 높은 점수를 얻을 수 있다고 생각합니다.

## 이 스크립트를 개선하는 방법
이것은 단지 경량 CNN이기 때문에, 성능이 제한적입니다. 성능을 향상시킬 수 있는 몇 가지 방법이 있습니다.

1. 원래 wav 파일을 사용합니다.
2. shop_audio를 사용하여 더 많은 '침묵' wav 파일을 만듭니다.
3. 심층 CNN을 구축하거나 RNN을 사용합니다.
4. 더 긴 에폭으로 훈련합니다.

In [1]:
import os
import numpy as np
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile

from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import keras

Using TensorFlow backend.


원래 샘플율은 16000이며, 데이터 크기를 줄이기 위해 8000으로 리샘플링 하겠습니다.

In [2]:
L = 16000
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()

# src folders
root_path = r'..'
out_path = r'.'
model_path = r'.'
train_data_path = os.path.join(root_path, 'input', 'train', 'audio')
test_data_path = os.path.join(root_path, 'input', 'test', 'audio')

In [3]:
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    # FFT is simmetrical, so we take just the first half
    # FFT is also complex, to we take just the real part (abs)
    vals = 2.0/N * np.abs(yf[0:N//2])
    return xf, vals

def log_specgram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio, fs=sample_rate, window='hann',
                                           nperseg=nperseg, noverlap=noverlap,
                                           detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32)+eps)
    

다음은 훈련 데이터 폴더 내의 모든 wav 파일을 잡을 수 있는 유틸리티 기능입니다.

In [4]:
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = f'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

__pad_audio__는 16000(1초) 미만의 오디오를 0으로 패딩하여 모두 동일한 길이를 갖도록 합니다.

__chop_audio__는 16000(예: 배경 노이즈 폴더의 wav 파일)보다 큰 오디오를 16000으로 잘라냅니다. 또한 'num' 매개 변수가 주어지면 하나의 큰 wav 파일 중에서 여러 개의 청크를 생성합니다.

__label_transform__은 레이블을 더미 값으로 변환합니다. 이것은 레이블을 예측하기 위해 softmax와 함께 사용됩니다.

In [5]:
def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0,0))
    
def chop_audio(samples, L=16000, num=20):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]
        
def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '__background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

다음으로, 위에서 선언한 함수를 사용하여 x_train 및 y_train을 생성합니다. label_index는 판다스에서  더미 값을 생성하기 위해 사용하는 색인입니다. 나중에 사용하기 위해 저장해야 합니다.

In [6]:
labels, fnames = list_wavs_fname(train_data_path)

new_sample_rate = 8000
y_train=[]
x_train=[]

for label, fname in zip(labels, fnames):
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        y_train.append(label)
        x_train.append(specgram)
x_train = np.array(x_train)
x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
y_train = label_transform(y_train)
label_index = y_train.columns.values
y_train = y_train.values
y_train = np.array(y_train)
del labels, fnames
gc.collect()

..\input\train\audio


C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


0

CNN은 아래와 같이 선언했습니다. 생성된 스펙그램은 모양이 (99, 81)이 되겠지만 Conv2D 레이어에 맞기 위해서는 형태를 바꿔야 합니다.

In [7]:
input_shape = (99, 81, 1)
nclass = 12
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(norm_inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

model.compile(optimizer=opt, loss=losses.binary_crossentropy)
model.summary()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)
model.fit(x_train, y_train, batch_size=16, validation_data=(x_valid, y_valid), epochs=3, shuffle=True, verbose=2)

model.save(os.path.join(model_path, 'cnn.model'))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 99, 81, 1)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 99, 81, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 98, 80, 8)         40        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 97, 79, 8)         264       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 39, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 39, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 37, 16)        1168

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

테스트 데이터가 너무 커서 RAM에 맞지 않아 하나씩 처리해야 합니다. Generator test_data_generator가 CNN에 연결할 테스트 Wav 파일 배치를 생성합니다.

In [ ]:
def test_data_generator(batch=16):
    fpaths = glob(os.path.join(test_data_path, '*wav'))
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        rate, samples = wavfile.read(path)
        samples = pad_audio(samples)
        resampled = signal.resample(samples, int(new_sample_rate / rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        imgs.append(specgram)
        fnames.append(path.split('\\')[-1])
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
        yield fames, imgs
    raise StopIteration()

훈련된 모델을 사용하여 테스트 데이터의 레이블을 예측합니다. 그러나 Kaggle은 테스트 데이터를 제공하지 않기 때문에 다음 섹션은 여기서 실행되지 않습니다.

In [ ]:
exit() # 
del x_train, y_train
gc.collect()

index = []
results = []
for fnames, imgs in test_data_generator(batch=32):
    predicts = model.predicts(imgs)
    predicts = np.argmax(predicts, axis=1)
    predicts = [label_index[p] for p in predicts]
    index.extend(fnames)
    results.extend(predicts)
    
df = pd.DataFrame(columns=['fname', 'label'])
df['fname'] = index
df['label'] = results
df.to_csv(os.path.join(out_path, 'sub.csv'), index=False)